In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.metrics import mean_absolute_error

In [ ]:
path= "/kaggle/input/insurance/insurance.csv"
data= pd.read_csv(path)
data.head()

In [ ]:
data.info()

In [ ]:
data.dtypes.value_counts()

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data.hist(figsize=(15,10))

In [ ]:
data.plot(kind="box")

In [ ]:
sns.heatmap(data.corr(), annot=True)

In [ ]:
sns.scatterplot(x= "age", y="charges", data= data, hue="region")

In [ ]:
sns.scatterplot(x= "bmi", y="charges", data= data, hue="sex")

In [ ]:
sns.scatterplot(x= "bmi", y="charges", data= data, size="smoker")

In [ ]:
x= data.drop("charges", axis= 1)
y= data.charges

x.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_tes= train_test_split(x,y,test_size= 0.3, random_state= 42)

In [ ]:
cat_cols= data.select_dtypes(include="object").columns
num_cols= data.select_dtypes(exclude="object").columns

cat_cols, num_cols

In [ ]:
num_cols= num_cols.drop("charges")

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [ ]:
x_train, x_test

In [ ]:
transformer= make_column_transformer( (OneHotEncoder(handle_unknown="ignore"), cat_cols),
                                      (MinMaxScaler(), num_cols),
                                     remainder= "passthrough"
                                    )

x_train= transformer.fit_transform(x_train)
x_test= transformer.transform(x_test)

In [ ]:
x_train[0].shape, x_train.shape

In [ ]:
tf.random.set_seed(42)

In [ ]:
model_1= keras.Sequential([
    keras.layers.Dense(4, input_shape=(11,)),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
], name="model-1")

In [ ]:
model_1.compile(loss= "mae",
              optimizer= tf.keras.optimizers.SGD(),
              metrics=["mae"]
             )

In [ ]:
model_1_history= model_1.fit(x_train, y_train, epochs=100)

In [ ]:
pd.DataFrame(model_1_history.history).plot()

In [ ]:
model_1.evaluate(x_test, y_tes)

# Overfitting 

In [ ]:
y_tes.shape, y_pred.shape

In [ ]:
y_pred= model_1.predict(x_test)
loss_1= mean_absolute_error(y_tes, tf.squeeze(y_pred)).numpy()
loss_1

> Model1 - training : 3493, testing : 5644

### Building model-2

In [ ]:
model_2= keras.Sequential([
    keras.layers.Dense(8, input_shape=(11,),  activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
], name="model-2")

model_2.compile(loss= "mae",
              optimizer= tf.keras.optimizers.Adam(lr=0.01),
              metrics=["mae"]
             )

model_2_history= model_2.fit(x_train, y_train, epochs=100)

In [ ]:
pd.DataFrame(model_2_history.history).plot()

In [ ]:
model_2.evaluate(x_test, y_tes)

In [ ]:
y_pred= model_2.predict(x_test)
loss_2= mean_absolute_error(y_tes, tf.squeeze(y_pred)).numpy()
loss_2

> Model-2 - Training : 2231, Testing : 1835

### Building Model-3

In [ ]:
model_3= keras.Sequential([
    keras.layers.Dense(8, input_shape=(11,),  activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
], name="model-3")

model_3.compile(loss= "mae",
              optimizer= tf.keras.optimizers.Adam(lr=0.01),
              metrics=["mae"]
             )

model_3_history= model_3.fit(x_train, y_train, epochs=120)

In [ ]:
pd.DataFrame(model_3_history.history).plot()

In [ ]:
model_3.evaluate(x_test, y_tes)

In [ ]:
y_pred= model_3.predict(x_test)
loss_3= mean_absolute_error(y_tes, tf.squeeze(y_pred)).numpy()
loss_3

> Model 3: Training : 1951, Testing : 1755

#### Same Model : Higher EPOCH

In [ ]:
model_4= keras.Sequential([
    keras.layers.Dense(8, input_shape=(11,),  activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
], name="model-4")

model_4.compile(loss= "mae",
              optimizer= tf.keras.optimizers.Adam(lr=0.01),
              metrics=["mae"]
             )

model_4_history= model_4.fit(x_train, y_train, epochs=150)

In [ ]:
pd.DataFrame(model_4_history.history).plot()

In [ ]:
model_4.evaluate(x_test, y_tes)

In [ ]:
y_pred= model_4.predict(x_test)
loss_4= mean_absolute_error(y_tes, tf.squeeze(y_pred)).numpy()
loss_4

In [ ]:
# Comparing all Models

res= [("model_1" , loss_1),
      ("model_2" , loss_2),
      ("model_3" , loss_3),
      ("model_4" , loss_4)
     ]
pd.DataFrame(res).set_index(0).plot(kind="bar", xlabel="models", ylabel="Losses")

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model_4, show_shapes = True)

### saving the Model

In [ ]:
model_4.save("medical_model_4.h5")

## Thanks for reading.... If u like this notebook, Please upvote it...
